In [ ]:
import os
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re
from tqdm.notebook import tqdm

In [ ]:
# Берём список файлов в нашей папке
files = [i for i in os.listdir() if i.endswith('txt')]
articles = defaultdict(list)

In [ ]:
# Создаём словарь, где ключ — газета, а значение — ссылка на статью
for file in files:
    key = file.replace('.txt', '')
    value = []
    with open(file, 'r') as f:
        for line in f.readlines():
            value.append(line.replace('\n', ''))
    articles[key] = value
# В txt файлах хранились ссылки на те статьи, которые релевантны поставленной задаче,
# к сожалению, они у меня не сохранились, но сохранились excel-файлы, в которых уже собрана информация
# по разным изданиям.

In [ ]:
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
url_page = '/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&ContestDirectionTenantId=&CompetitionId=9c0a9f89-727c-453e-a7b7-555cded82f57&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&page='
main = 'https://xn--80afcdbalict6afooklqi5o.xn--p1ai'

In [ ]:
def to_year(x):
    try:
        a = int(re.search('20\d{2}', x).group())
    except:
        a = 0
    return a

In [ ]:
article_content = [] # Здесь мы будем собирать содержание статей

# Парсим заголовки и названия статей в rg
for value in articles['rg']:
    page = requests.get(value, timeout=10, headers=HEADERS)
    soup = BeautifulSoup(page.text, 'lxml')
    title = soup.find_all('title')[0].text
    text = soup.find_all('div', {'class': 'b-material-wrapper__text'})[0].text
    text = text.replace('\n-', '').replace('\n', '').replace('\xa0', '').replace('\r', '')
    article_content.append([value, title, text])
    sleep(3)

rg = pd.DataFrame(article_content, columns=['url', 'title', 'content'])
rg['year'] = rg['url'].apply(lambda x: to_year(x))
rg.to_excel('rg_articles.xlsx', index=False)

In [ ]:
article_content = []

# Парсим заголовки и названия статей в gazeta
for value in tqdm(articles['gazeta']):
    page = requests.get(value, timeout=20, headers=HEADERS)
    soup = BeautifulSoup(page.text, 'lxml')
    title = soup.find_all('title')[0].text.replace('Новости', '').replace('Газета.Ru', '').replace('|', '').replace('  ', '')
    try:
        text = soup.find_all('div', {'class': 'article-text-body'})[0].text
    except:
        try:
            text = soup.find_all('div', {'class': 'article_text txt_1'})[0].text
        except:
            try:
                text = soup.find_all('p', {'class': 'txt_3 mb20 mt10'})[0].text
            except:
                try:
                    text = soup.find_all('h2', {'class': 'b_photogallery-intro mt15 mb10'})[0].text
                except:
                    text = soup.find_all('div', {'class': 'article_text txt_1 pl20'})[0].text
    text = text.replace('\n-', '').replace('\n', '').replace('\xa0', '').replace('\r', '')
    article_content.append([value, title[1:-2], text])
    sleep(4)

gazeta = pd.DataFrame(article_content, columns=['url', 'title', 'content'])
gazeta['year'] = gazeta['url'].apply(lambda x: to_year(x))

gazeta.loc[54, 'year'] = 2014
gazeta.loc[59, 'year'] = 2014
gazeta.loc[121, 'year'] = 2015
gazeta.loc[270, 'year'] = 2017
gazeta.loc[279, 'year'] = 2018
gazeta.loc[428, 'year'] = 2020

gazeta.to_excel('gazeta_articles.xlsx', index=False)

In [ ]:
article_content = []

# Парсим заголовки и названия статей в mk
for value in tqdm(articles['mk'][20:]):
    page = requests.get(value, timeout=10, headers=HEADERS)
    soup = BeautifulSoup(page.text, 'lxml')
    title = soup.find_all('title')[0].text
    title = title.replace('МК', '')
    try:
        text = soup.find_all('div', {'class': 'article__body'})[0].text
        text = text.split('window.MKInreadsInArticle')[0].replace('  ', '')
    except:
        text = soup.find_all('div', {'class': 'inread-content'})[0].text
    text = text.replace('\n-', '').replace('\n', '').replace('\xa0', '').replace('\r', '').replace('<br/>', '')
    article_content.append([value, title, text])
    sleep(3)

mk = pd.DataFrame(article_content, columns=['url', 'title', 'content'])
mk['year'] = mk['url'].apply(lambda x: to_year(x))
mk.to_excel('mk_articles.xlsx', index=False)

In [ ]:
article_content = []

# Парсим заголовки и названия статей в iz
for i in range(10, 21):
    year = '20' + str(i)
    for value in tqdm(articles['iz' + str(i)]):
        page = requests.get(value, timeout=10, headers=HEADERS)
        soup = BeautifulSoup(page.text, 'lxml')
        title = soup.find_all('title')[0].text
        text = soup.find_all('div', {'class': 'text-article__inside'})[0].text
        text = text.replace('\n-', '').replace('\n', '').replace('\xa0', '').replace('\r', '').replace('\t', '')
        text = text.split('Поделиться')[0]
        article_content.append([value, title, text, int(year)])
        sleep(3)

iz = pd.DataFrame(article_content, columns=['url', 'title', 'content', 'year'])
iz.to_excel('iz_articles.xlsx', index=False)

In [ ]:
article_content = []

# Парсим заголовки и названия статей в ng
for i in range(10, 21):
    year = '20' + str(i)
    for value in tqdm(articles['ng' + str(i)][1:]):
        page = requests.get(value, timeout=10, headers=HEADERS)
        soup = BeautifulSoup(page.text, 'lxml')
        try:
            title = soup.find_all('title')[0].text
        except:
            title = soup.find_all('h1')[0].text
        try:
            text = soup.find_all('div', {'class': 'news_detail_content'})[0].text
        except:
            text = soup.find_all('article', {'class': 'typical'})[0].text
        text = text.replace('\n-', '').replace('\n', '').replace('\xa0', '').replace('\r', '').replace('\t', '')
        article_content.append([value, title, text, int(year)])
        sleep(3)

ng = pd.DataFrame(article_content, columns=['url', 'title', 'content', 'year'])
ng.to_excel('ng_articles.xlsx', index=False)

In [ ]:
article_content = []

# Парсим заголовки и названия статей в rbk
for i in range(10, 21):
    if i == 13:
        None
    else:
        year = '20' + str(i)
        for value in tqdm(articles['rbk' + str(i)][1:]):
            page = requests.get(value, timeout=10, headers=HEADERS)
            soup = BeautifulSoup(page.text, 'lxml')
            title = soup.find_all('title')[0].text
            soup.find_all('div', {'class': 'article__text article__text_free'})[0].text
            text = text.replace('\n-', '').replace('\n', '').replace('\xa0', '').replace('\r', '').replace('\t', '')
            article_content.append([value, title, text, int(year)])
            sleep(3)

rbk = pd.DataFrame(article_content, columns=['url', 'title', 'content', 'year'])
rbk.to_excel('rbk_articles.xlsx', index=False)